# Define Data

In [ ]:
from IPython.display import clear_output
!pip install autogluon --user
clear_output()

import numpy as np
import pandas as pd 
import os 
import random
import warnings
warnings.filterwarnings(action='ignore')

from autogluon.tabular import TabularPredictor

TRAIN_PATH = "../input/spaceship-titanic/train.csv"
TEST_PATH = "../input/spaceship-titanic/test.csv"
SAMPLE_SUBISSION_PATH = "../input/spaceship-titanic/sample_submission.csv"
SUBMISSION_PATH = "submission.csv"
TARGET = "Transported"

SEED = 2022
def seed_everything(seed=SEED):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
seed_everything()

MODEL_EVAL_METRIC = 'accuracy'
MODEL_TIME_LIMIT = 60*30
MODEL_SAVE_PATH = "autogluon_model/"
MODEL_VERBOSE = 3
MODEL_PRESETS = "best_quality"

# Build Model

In [ ]:
# import module we'll need to import our custom module
from shutil import copyfile

# copy our file into the working directory (make sure it has .py suffix)
copyfile(src = "../input/autosplitfeatureutil/auto_splited_feature_util.py", dst = "../working/auto_splited_feature_util.py")

from auto_splited_feature_util import *

In [ ]:
train = pd.read_csv(TRAIN_PATH)

COL = "Cabin"
SPLIT_SIZE = 3
SEP = "/"

train = addSplitedColumn(train,COL,SPLIT_SIZE,SEP)

predictor = TabularPredictor(
    label=TARGET, 
    eval_metric=MODEL_EVAL_METRIC, 
    path=MODEL_SAVE_PATH, 
    verbosity=MODEL_VERBOSE).fit(
    train, 
    presets=MODEL_PRESETS, 
    time_limit=MODEL_TIME_LIMIT
)

# Predict Data

In [ ]:
test = pd.read_csv(TEST_PATH)
COL = "Cabin"
SPLIT_SIZE = 3
SEP = "/"

test = addSplitedColumn(test,COL,SPLIT_SIZE,SEP)

pred_test = predictor.predict_proba(test, as_multiclass=False)

sub = pd.read_csv(SAMPLE_SUBISSION_PATH)
sub[TARGET] = (pred_test > 0.5).astype(bool)
sub.to_csv(SUBMISSION_PATH,index=False)
sub.head()